In [ ]:
import os
import re

with open("visit to dancing rabbit 2012 condensed.txt", 'r') as f:
    text = f.read()
trimmed = re.sub('\n#+', '#', text)

In [ ]:
def put_header_symbol(match):
    return '#' + match.group(0).replace('#','').strip()

trimmed = re.sub('^.+#', put_header_symbol, trimmed, flags = re.MULTILINE)

In [ ]:
from collections import OrderedDict

trimmed = re.sub('\n+', '\n', trimmed)
sections = trimmed[1:].split('#')
len(sections)
sect_dict = OrderedDict()
for s in sections:
    (title, body) = s.split('\n', 1)
    sect_dict[title.strip()] = body.strip()
    
for k in sect_dict:
    print(k)

In [ ]:
xml_header = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE concept PUBLIC "-//OASIS//DTD DITA Concept//EN" "concept.dtd">
<concept id="{}">'''
xml_footer = '''</conbody>
    </concept>'''
    
outputs = []
for s in sect_dict:
    title_id = s.lower().replace(' ', '_')
    cur_header = xml_header.format(title_id) + '<title>{}</title>\n'.format(s)
    cur_body = '<conbody>\n<p>' + sect_dict[s].replace('\n', '</p>\n<p>') + '</p></conbody>'
    cur_output = cur_header + '\n' + cur_body + xml_footer
    xml_escape_dict = {'"': '&quot;', '&': '&amp;', "'": '&apos;', '<': '&lt;', '>': '&gt;'}
    outputs.append(cur_output)    

In [ ]:
from bs4 import BeautifulSoup

for out in outputs:
    bs = BeautifulSoup(out, 'xml')
    filename = bs.concept.get('id') + '.dita'
    with open(filename, 'w') as f:
        f.write(bs.prettify())

In [ ]:
# Producing the DITA map.

import glob

map_blank = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE map PUBLIC "-//OASIS//DTD DITA Map//EN" "map.dtd">
<map>
    <title>My Visit to Dancing Rabbit Ecovillage</title>
    {}
</map>'''

topicref_blank = '<topicref href="{}"/>'
topicref_text = ''
for k in sect_dict:
    topicref_text += topicref_blank.format(k.lower().replace(' ', '_')+'.dita') + '\n'

print(map_blank.format(topicref_text))